<a href="https://colab.research.google.com/github/CliodhnaNiShe/MIS41160/blob/main/Tutorial9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import math

import numpy as np

import networkx as nx

import matplotlib.pyplot as plt

In [5]:
dist = [[0, 28, 31, 20, 25, 34],
        [28, 0, 21, 29, 26, 20],
        [31, 21, 0, 38, 20, 32],
        [20, 29, 38, 0, 30, 27],
        [25, 26, 20, 30, 0, 26],
        [34, 20, 32, 27, 25]]

## Randomised Heuristic

Randomly shuffle the list of nodes to create a tour.

In [6]:
import random

def tsp_randomized(dist_matrix, iterations=1):

    n = len(dist_matrix)
    best_tour = None
    best_cost = float('inf')

    for _ in range(iterations):
        # Generate a random tour
        tour = list(range(n))
        random.shuffle(tour)
        tour.append(tour[0])  # Return to the starting node

        # Calculate the cost of the tour
        total_cost = 0
        for i in range(n):
            total_cost += dist_matrix[tour[i]][tour[i + 1]]

        # Update the best tour if this one is better
        if total_cost < best_cost:
            best_cost = total_cost
            best_tour = tour

    return best_tour, best_cost


# Distance matrix
dist = [
    [0, 28, 31, 20, 25, 34],
    [28, 0, 21, 29, 26, 20],
    [31, 21, 0, 38, 20, 32],
    [20, 29, 38, 0, 30, 27],
    [25, 26, 20, 30, 0, 26],
    [34, 20, 32, 27, 25, 0]
]

# Solve the TSP using the randomized heuristic
iterations = 1
best_tour, best_cost = tsp_randomized(dist, iterations)

# Print the results
print(f"Best Tour (after {iterations} iterations):", best_tour)
print("Best Total Cost:", best_cost)


Best Tour (after 1 iterations): [3, 2, 4, 5, 1, 0, 3]
Best Total Cost: 152


We can repeat this for n iterations and keep the result, to make this a better heuristic

In [7]:
import random

def tsp_randomized(dist_matrix, iterations=1):

    n = len(dist_matrix)
    best_tour = None
    best_cost = float('inf')

    for _ in range(iterations):
        # Generate a random tour
        tour = list(range(n))
        random.shuffle(tour)
        tour.append(tour[0])  # Return to the starting node

        # Calculate the cost of the tour
        total_cost = 0
        for i in range(n):
            total_cost += dist_matrix[tour[i]][tour[i + 1]]

        # Update the best tour if this one is better
        if total_cost < best_cost:
            best_cost = total_cost
            best_tour = tour

    return best_tour, best_cost


# Distance matrix
dist = [
    [0, 28, 31, 20, 25, 34],
    [28, 0, 21, 29, 26, 20],
    [31, 21, 0, 38, 20, 32],
    [20, 29, 38, 0, 30, 27],
    [25, 26, 20, 30, 0, 26],
    [34, 20, 32, 27, 25, 0]
]

# Solve the TSP using the randomized heuristic
iterations = 100
best_tour, best_cost = tsp_randomized(dist, iterations)

# Print the results
print(f"Best Tour (after {iterations} iterations):", best_tour)
print("Best Total Cost:", best_cost)


Best Tour (after 100 iterations): [4, 0, 3, 5, 1, 2, 4]
Best Total Cost: 133


## Nearest neighbour heuristic

Start at an initial node, and chose the closest unvisited node until all nodes are visited

In [8]:
def tsp_nearest_neighbor(dist_matrix, start_node=0):

    n = len(dist_matrix)
    visited = [False] * n
    tour = [start_node]
    total_cost = 0

    current_node = start_node
    visited[current_node] = True

    for _ in range(n - 1):
        # Find the nearest unvisited node
        nearest_node = None
        min_distance = float('inf')

        for next_node in range(n):
            if not visited[next_node] and dist_matrix[current_node][next_node] < min_distance:
                nearest_node = next_node
                min_distance = dist_matrix[current_node][next_node]

        # Update the tour and total cost
        tour.append(nearest_node)
        total_cost += min_distance
        visited[nearest_node] = True
        current_node = nearest_node

    # Return to the starting node
    total_cost += dist_matrix[current_node][start_node]
    tour.append(start_node)

    return tour, total_cost


# Solve the TSP using the nearest neighbor heuristic
start_node = 0
tour, total_cost = tsp_nearest_neighbor(dist, start_node)

# Print the results
print("Tour:", tour)
print("Total cost:", total_cost)

Tour: [0, 3, 5, 1, 2, 4, 0]
Total cost: 133


Think about how changing the initial strarting node affects the result, both the tour and the objective function value

In [9]:
# Compare costs for different starting nodes
for start_node in range(len(dist)):
    tour, total_cost = tsp_nearest_neighbor(dist, start_node)
    print(f"Starting at node {start_node}: Tour = {tour}, Total Cost = {total_cost}")

Starting at node 0: Tour = [0, 3, 5, 1, 2, 4, 0], Total Cost = 133
Starting at node 1: Tour = [1, 5, 4, 2, 0, 3, 1], Total Cost = 145
Starting at node 2: Tour = [2, 4, 0, 3, 5, 1, 2], Total Cost = 133
Starting at node 3: Tour = [3, 0, 4, 2, 1, 5, 3], Total Cost = 133
Starting at node 4: Tour = [4, 2, 1, 5, 3, 0, 4], Total Cost = 133
Starting at node 5: Tour = [5, 1, 2, 4, 0, 3, 5], Total Cost = 133


## Greedy Heuristic

Adds the shortest available edge to the tour iteratively while avoiding premature cycles and ensuring no node has more than two connections (except for closing the tour)

In [10]:
def tsp_greedy(dist_matrix):

    n = len(dist_matrix)

    # List to track edges in the solution
    edges = []
    degrees = [0] * n  # Degree of each node
    total_cost = 0

    # Sort all edges by their distance
    all_edges = [(i, j, dist_matrix[i][j]) for i in range(n) for j in range(i + 1, n)]
    all_edges.sort(key=lambda x: x[2])

    # Helper function to find the set of a node (to avoid cycles)
    parent = list(range(n))

    def find(node):
        if parent[node] != node:
            parent[node] = find(parent[node])
        return parent[node]

    def union(node1, node2):
        parent[find(node1)] = find(node2)

    # Add edges while avoiding premature cycles and ensuring degrees are valid
    for i, j, cost in all_edges:
        if degrees[i] < 2 and degrees[j] < 2 and find(i) != find(j):
            edges.append((i, j))
            total_cost += cost
            degrees[i] += 1
            degrees[j] += 1
            union(i, j)

        # Stop when we have n edges to form a cycle
        if len(edges) == n - 1:
            break

    # Close the tour
    start_node, end_node = None, None
    for node in range(n):
        if degrees[node] == 1:
            if start_node is None:
                start_node = node
            else:
                end_node = node
                break
    edges.append((start_node, end_node))
    total_cost += dist_matrix[start_node][end_node]

    # Extract the tour from the edges
    tour = [start_node]
    current_node = start_node
    while len(tour) < n:
        for edge in edges:
            if edge[0] == current_node and edge[1] not in tour:
                tour.append(edge[1])
                current_node = edge[1]
                break
            elif edge[1] == current_node and edge[0] not in tour:
                tour.append(edge[0])
                current_node = edge[0]
                break
    tour.append(start_node)

    return tour, total_cost

# Solve the TSP using the greedy heuristic
tour, total_cost = tsp_greedy(dist)
print("Greedy Tour:", tour)
print("Total Cost:", total_cost)


Greedy Tour: [3, 0, 4, 2, 1, 5, 3]
Total Cost: 133


## Testing heuristics on benchmark instances

In [11]:
import math

# Node coordinates
coordinates = [
    (16.47, 96.10), (16.47, 94.44), (20.09, 92.54), (22.39, 93.37),
    (25.23, 97.24), (22.00, 96.05), (20.47, 97.02), (17.20, 96.29),
    (16.30, 97.38), (14.05, 98.12), (16.53, 97.38), (21.52, 95.59),
    (19.41, 97.13), (20.09, 94.55)
]

# Function to calculate Euclidean distance
def euclidean_distance(p1, p2):
    return round(math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2), 2)

# Create the distance matrix
n = len(coordinates)
distance_matrix = [[0] * n for _ in range(n)]

for i in range(n):
    for j in range(n):
        if i != j:
            distance_matrix[i][j] = euclidean_distance(coordinates[i], coordinates[j])

# Print the distance matrix
print("Distance Matrix:")
for row in distance_matrix:
    print(row)


Distance Matrix:
[0, 1.66, 5.08, 6.52, 8.83, 5.53, 4.1, 0.75, 1.29, 3.15, 1.28, 5.08, 3.12, 3.94]
[1.66, 0, 4.09, 6.02, 9.2, 5.76, 4.76, 1.99, 2.94, 4.4, 2.94, 5.18, 3.98, 3.62]
[5.08, 4.09, 0, 2.45, 6.96, 4.0, 4.5, 4.73, 6.15, 8.22, 6.01, 3.37, 4.64, 2.01]
[6.52, 6.02, 2.45, 0, 4.8, 2.71, 4.12, 5.96, 7.29, 9.6, 7.1, 2.38, 4.8, 2.59]
[8.83, 9.2, 6.96, 4.8, 0, 3.44, 4.77, 8.09, 8.93, 11.21, 8.7, 4.06, 5.82, 5.8]
[5.53, 5.76, 4.0, 2.71, 3.44, 0, 1.81, 4.81, 5.85, 8.22, 5.63, 0.66, 2.81, 2.43]
[4.1, 4.76, 4.5, 4.12, 4.77, 1.81, 0, 3.35, 4.19, 6.51, 3.96, 1.77, 1.07, 2.5]
[0.75, 1.99, 4.73, 5.96, 8.09, 4.81, 3.35, 0, 1.41, 3.64, 1.28, 4.38, 2.36, 3.37]
[1.29, 2.94, 6.15, 7.29, 8.93, 5.85, 4.19, 1.41, 0, 2.37, 0.23, 5.52, 3.12, 4.73]
[3.15, 4.4, 8.22, 9.6, 11.21, 8.22, 6.51, 3.64, 2.37, 0, 2.59, 7.89, 5.45, 7.02]
[1.28, 2.94, 6.01, 7.1, 8.7, 5.63, 3.96, 1.28, 0.23, 2.59, 0, 5.3, 2.89, 4.55]
[5.08, 5.18, 3.37, 2.38, 4.06, 0.66, 1.77, 4.38, 5.52, 7.89, 5.3, 0, 2.61, 1.77]
[3.12, 3.98, 4.64, 4

Randomised Heuristic

In [12]:
# Solve the TSP using the randomized heuristic
iterations = 1
best_tour, best_cost = tsp_randomized(distance_matrix, iterations)

# Print the results
print(f"Best Tour (after {iterations} iterations):", best_tour)
print("Best Total Cost:", best_cost)

Best Tour (after 1 iterations): [2, 7, 1, 12, 8, 6, 4, 9, 10, 11, 5, 0, 13, 3, 2]
Best Total Cost: 57.05


In [13]:
# Solve the TSP using the randomized heuristic
iterations = 100
best_tour, best_cost = tsp_randomized(distance_matrix, iterations)

# Print the results
print(f"Best Tour (after {iterations} iterations):", best_tour)
print("Best Total Cost:", best_cost)

Best Tour (after 100 iterations): [3, 13, 2, 4, 12, 1, 10, 9, 8, 0, 7, 5, 11, 6, 3]
Best Total Cost: 42.66


Try for more iterations - 1000 or 10000

Nearest Neighbour Heuristic

In [14]:
# Solve the TSP using the nearest neighbor heuristic
start_node = 0
tour, total_cost = tsp_nearest_neighbor(distance_matrix, start_node)


# Print the results
print("Tour:", tour)
print("Total cost:", total_cost)

Tour: [0, 7, 10, 8, 9, 1, 13, 11, 5, 6, 12, 2, 3, 4, 0]
Total cost: 38.68


In [15]:
# Compare costs for different starting nodes
for start_node in range(len(distance_matrix)):
    tour, total_cost = tsp_nearest_neighbor(distance_matrix, start_node)
    print(f"Starting at node {start_node}: Tour = {tour}, Total Cost = {total_cost}")

Starting at node 0: Tour = [0, 7, 10, 8, 9, 1, 13, 11, 5, 6, 12, 2, 3, 4, 0], Total Cost = 38.68
Starting at node 1: Tour = [1, 0, 7, 10, 8, 9, 12, 6, 11, 5, 13, 2, 3, 4, 1], Total Cost = 36.129999999999995
Starting at node 2: Tour = [2, 13, 11, 5, 6, 12, 7, 0, 10, 8, 9, 1, 3, 4, 2], Total Cost = 36.49
Starting at node 3: Tour = [3, 11, 5, 6, 12, 7, 0, 10, 8, 9, 1, 13, 2, 4, 3], Total Cost = 34.7
Starting at node 4: Tour = [4, 5, 11, 6, 12, 7, 0, 10, 8, 9, 1, 13, 2, 3, 4], Total Cost = 31.21
Starting at node 5: Tour = [5, 11, 6, 12, 7, 0, 10, 8, 9, 1, 13, 2, 3, 4, 5], Total Cost = 31.209999999999997
Starting at node 6: Tour = [6, 12, 7, 0, 10, 8, 9, 1, 13, 11, 5, 3, 2, 4, 6], Total Cost = 35.400000000000006
Starting at node 7: Tour = [7, 0, 10, 8, 9, 1, 13, 11, 5, 6, 12, 2, 3, 4, 7], Total Cost = 37.94
Starting at node 8: Tour = [8, 10, 0, 7, 1, 13, 11, 5, 6, 12, 2, 3, 4, 9, 8], Total Cost = 38.65
Starting at node 9: Tour = [9, 8, 10, 0, 7, 1, 13, 11, 5, 6, 12, 2, 3, 4, 9], Total Cost 